### Ownership of Locks

Ownership of locks has changed since OpenMP 2.5. In OpenMP 2.5, locks are owned  by threads; so a lock released by the `omp_unset_lock` routine must be  owned by the same thread executing the routine.  Beginning with OpenMP 3.0, locks are owned  by task regions; so a lock released by the `omp_unset_lock` routine in  a task region must be owned by the same task region.

This change in ownership requires extra care when using locks. The following program  is conforming in OpenMP 2.5 because the thread that releases the lock `lck`  in the parallel region is the same thread that acquired the lock in the sequential  part of the program (master thread of parallel region and the initial thread are  the same). However, it is not conforming beginning with OpenMP 3.0, because the task  region that releases the lock `lck` is different from the task region that  acquires the lock.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: lock_owner.1c
* type: C
*/
#include <stdlib.h>
#include <stdio.h>
#include <omp.h>

int main()
{
  int x;
  omp_lock_t lck;

  omp_init_lock (&lck);
  omp_set_lock (&lck);
  x = 0;

#pragma omp parallel shared (x)
  {
    #pragma omp master
      {
        x = x + 1;
        omp_unset_lock (&lck);
      }

    /* Some more stuff. */
  }
  omp_destroy_lock (&lck);
  return 0;
}



In [ ]:

! name: lock_owner.1f
! type: F-fixed
        program lock
        use omp_lib
        integer :: x
        integer (kind=omp_lock_kind) :: lck

        call omp_init_lock (lck)
        call omp_set_lock(lck)
        x = 0

!$omp parallel shared (x)
!$omp master
        x = x + 1
        call omp_unset_lock(lck)
!$omp end master

!       Some more stuff.
!$omp end parallel

        call omp_destroy_lock(lck)
        end

